In [1]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/Users/colinbull/.pyenv/versions/3.11.4/lib/python311.zip', '/Users/colinbull/.pyenv/versions/3.11.4/lib/python3.11', '/Users/colinbull/.pyenv/versions/3.11.4/lib/python3.11/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.11/lib/python3.11/site-packages']


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [2]:
import src.pipeline.pre_processing as pre_processing
import src.pipeline.input_schemas as input_schemas
import src.pipeline.bfr as BFR
import pandas as pd
import numpy as np
import time
import glob
import os

In [3]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

# files = glob.glob("output/pre-processing/*")
# for f in files:
#     os.remove(f)

In [4]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [5]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [6]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

,Total Internal Floor Area,Age Average Score,Building Age
URN,,,
100150,2803.0,48.358188,1988
100162,2105.0,133.162945,1880
100164,2934.0,97.0,1925
100166,2040.0,91.705882,1947
105304,1602.0,35.752809,1985
...,...,...,...
144913,3111.0,16.704275,2008
144917,2620.0,78.412214,1960
105623,3382.0,7.0,2015


## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [7]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')


In [8]:
#census.to_csv('output/pre-processing/census.csv')
census

,region_name,district_administrative_name,ward_name,Full time girls Year group 12,Full time girls Year group 13,Full time boys Year group 12,Full time boys Year group 13,Number of early year pupils (years E1 and E2),Number of nursery pupils (years N1 and N2),Number of pupils,...,Total Number of Auxiliary Staff (Headcount),Total School Workforce (Full-Time Equivalent),Total Number of Teachers in the Leadership Group (Full-time Equivalent),Total Number of Teachers (Full-Time Equivalent),Total Number of Teaching Assistants (Full-Time Equivalent),NonClassroomSupportStaffFTE,Total Number of Auxiliary Staff (Full-Time Equivalent),Teachers with Qualified Teacher Status (%) (Headcount),TotalPupilsNursery,TotalPupilsSixthForm
URN,,,,,,,,,,,,,,,,,,,,,
141334,East Midlands,Nottingham,Bilborough,0.0,0.0,0.0,0.0,0.0,0.0,325.0,...,13.0,34.17,2.64,13.11,10.29,4.82,5.95,100.000000,0.0,0.0
141396,East Midlands,Nottingham,Aspley,0.0,0.0,0.0,0.0,0.0,57.0,642.0,...,34.0,82.47,4.00,34.00,29.55,10.13,8.79,100.000000,57.0,0.0
141397,East Midlands,Nottingham,Bilborough,0.0,0.0,0.0,0.0,0.0,44.0,500.0,...,27.0,72.81,4.24,24.55,31.84,6.55,9.87,100.000000,44.0,0.0
142223,East Midlands,Nottingham,Wollaton West,0.0,0.0,0.0,0.0,0.0,70.0,1117.0,...,47.0,99.66,4.16,47.12,33.07,6.57,12.90,100.000000,70.0,0.0
144396,East Midlands,Nottingham,Bulwell,0.0,0.0,0.0,0.0,0.0,21.0,217.0,...,11.0,25.57,2.00,11.39,7.36,4.00,2.82,100.000000,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104642,North West,Liverpool,Church,0.0,0.0,0.0,0.0,0.0,0.0,421.0,...,18.0,34.47,3.60,15.80,6.27,5.22,7.18,100.000000,0.0,0.0
104643,North West,Liverpool,Cressington,0.0,0.0,0.0,0.0,0.0,28.0,432.0,...,19.0,39.89,3.00,17.40,11.74,3.34,7.41,100.000000,28.0,0.0
104645,North West,Liverpool,Tuebrook and Stoneycroft,0.0,0.0,0.0,0.0,0.0,0.0,237.0,...,10.0,26.47,3.00,12.40,6.78,3.19,4.10,92.307692,0.0,0.0


## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [9]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [10]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

,EHC plan,SEN support,Percentage SEN,Percentage with EHC,Percentage without EHC,Percentage Primary Need SPLD,Percentage Primary Need MLD,Percentage Primary Need SLD,Percentage Primary Need PMLD,Percentage Primary Need SEMH,Percentage Primary Need SLCN,Percentage Primary Need HI,Percentage Primary Need VI,Percentage Primary Need MSI,Percentage Primary Need PD,Percentage Primary Need ASD,Percentage Primary Need OTH
URN,,,,,,,,,,,,,,,,,
100000,8.0,59.0,24.723247,2.952030,21.771218,0.738007,1.476015,0.0,0.000000,3.321033,11.439114,0.738007,0.000000,0.738007,0.000000,3.321033,0.369004
100001,0.0,22.0,2.976996,0.000000,2.976996,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100002,0.0,22.0,8.178439,0.000000,8.178439,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100003,0.0,145.0,13.875598,0.000000,13.875598,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100005,2.0,23.0,18.382353,1.470588,16.911765,0.000000,0.000000,0.0,0.735294,0.000000,0.000000,0.000000,0.000000,0.000000,0.735294,16.176471,0.735294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,3.0,2.0,12.195122,7.317073,4.878049,4.878049,0.000000,0.0,0.000000,0.000000,2.439024,0.000000,0.000000,0.000000,0.000000,4.878049,0.000000
149632,58.0,136.0,15.027111,4.492641,10.534469,2.401239,1.161890,0.0,0.000000,1.549187,1.936483,0.619675,0.309837,0.077459,0.542215,2.013943,1.781565
149633,0.0,7.0,8.139535,0.000000,8.139535,2.325581,1.162791,0.0,0.000000,1.162791,1.162791,0.000000,0.000000,0.000000,0.000000,1.162791,1.162791


## KS2 and KS4 processing

In [11]:
ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')

In [12]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

,Ks2Progress
URN,
100000,0.5
136807,13.0
139837,20.7
140686,-1.8
100008,5.5
...,...
143723,-28.9
146250,-20.5
141993,0.0


In [13]:
ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')

In [14]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

,AverageAttainment,Progress8Measure,Progress8Banding
URN,,,
100053,50.3,-0.16,Average
100054,65.8,0.77,Well above average
100052,44.6,-0.03,Average
100092,0.5,-2.22,Well below average
100049,41.7,-0.28,Below average
...,...,...,...
112465,0.5,-1.53,Well below average
137269,50.2,0.36,Average
112393,40.3,-0.11,Average


## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [15]:
academy_ar = pre_processing.prepare_aar_data('data/academy_ar.xlsx')
central_services = pre_processing.prepare_central_services_data('data/academy_ar.xlsx')

In [16]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

,Academy UPIN,Lead UPIN,Date left or closed if in period,Date joined or opened if in period,London Weighting,PFI School,Income_Other DFE grants,Income_SEN funding,Income_Pre Post 16,Income_Other grants,...,Catering Expenses,Occupation Costs,Total Costs of Supplies and Services,Total Costs of Educational Supplies,Costs of Brought in Professional Services,Total Expenditure,Trust Balance,Financial Position,Trust Financial Position,Is PFI
URN,,,,,,,,,,,,,,,,,,,,,
138623,111443,137157,<NA>,<NA>,Neither,Non-PFI school,7967000.0,153000.0,262000.0,403000.0,...,693000.0,1163000.0,1167000.0,750000.0,204000.0,10949000.0,-1899000.0,Deficit,Deficit,False
138630,111451,138199,<NA>,<NA>,Neither,Non-PFI school,6342000.0,222000.0,7000.0,327000.0,...,95000.0,326000.0,1398000.0,1038000.0,209000.0,7724000.0,2880000.0,Deficit,Surplus,False
143005,111453,135112,<NA>,<NA>,Neither,Non-PFI school,2798000.0,162000.0,63000.0,36000.0,...,43000.0,243000.0,409000.0,159000.0,122000.0,2949000.0,1985000.0,Surplus,Surplus,False
136296,111710,135428,<NA>,<NA>,Neither,Non-PFI school,7685000.0,323000.0,215000.0,470000.0,...,422000.0,771000.0,1331000.0,952000.0,71000.0,10613000.0,-1544000.0,Deficit,Deficit,False
137982,113087,136879,<NA>,<NA>,Neither,Non-PFI school,8021000.0,93000.0,45000.0,95000.0,...,104000.0,398000.0,1341000.0,428000.0,106000.0,10356000.0,-1232000.0,Deficit,Deficit,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150026,164644,151923,<NA>,2023-07-01,Neither,Non-PFI school,227000.0,18000.0,2000.0,0.0,...,18000.0,25000.0,15000.0,10000.0,3000.0,252000.0,341000.0,Deficit,Surplus,False
150133,164745,136351,<NA>,2023-07-01,Neither,Non-PFI school,671000.0,183000.0,44000.0,0.0,...,5000.0,26000.0,38000.0,14000.0,7000.0,480000.0,1768000.0,Surplus,Surplus,False
150134,164746,135065,<NA>,2023-08-01,Neither,Non-PFI school,83000.0,127000.0,0.0,0.0,...,0.0,7000.0,0.0,0.0,0.0,201000.0,5380000.0,Surplus,Surplus,False


In [17]:
central_services

,Income_Other DFE grants,Income_SEN funding,Income_Pre Post 16,Income_Other grants,Income_Government source,Income_Academies,Income_Non government,Income_Facilities and services,Income_Catering services,Income_Receipts supply teacher insurance,...,Staff Total,Maintenance & Improvement,Premises,Catering Exp,Occupation,Supplies and Services,Educational Supplies,Brought in Professional Services,Total Expenditure,Financial Position
Trust UPIN,,,,,,,,,,,,,,,,,,,,,
122824,740000.0,5000.0,0.0,0.0,0.0,0.0,0.0,37000.0,3000.0,0.0,...,678000.0,15000.0,15000.0,0.0,9000.0,1434000.0,451000.0,791000.0,2131000.0,Deficit
122836,0.0,0.0,0.0,444000.0,0.0,0.0,444000.0,0.0,0.0,0.0,...,17582000.0,6584000.0,10602000.0,25000.0,826000.0,4053000.0,94000.0,2430000.0,33063000.0,Deficit
124263,0.0,0.0,0.0,-239000.0,0.0,0.0,-189000.0,0.0,0.0,0.0,...,3880000.0,19000.0,336000.0,-7000.0,2222000.0,526000.0,185000.0,88000.0,6964000.0,Deficit
126463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Surplus
133307,0.0,0.0,113000.0,42000.0,0.0,0.0,42000.0,0.0,0.0,0.0,...,1718000.0,721000.0,771000.0,0.0,68000.0,284000.0,55000.0,91000.0,2841000.0,Deficit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21000.0,0.0,0.0,...,96000.0,0.0,0.0,0.0,0.0,21000.0,0.0,20000.0,117000.0,Deficit
164130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Surplus
164242,175000.0,0.0,11000.0,2000.0,0.0,0.0,2000.0,0.0,0.0,0.0,...,78000.0,0.0,0.0,0.0,0.0,211000.0,27000.0,180000.0,289000.0,Deficit


Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [18]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [19]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentStatus (code),EstablishmentStatus (name),OpenDate,CloseDate,...,BoardingEstablishment (name),PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,OfstedRating (name),MSOA (code),LSOA (code),LA Establishment Number,Has Nursery,Has Sixth Form
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,3614,The Aldgate School,2,Voluntary aided school,1,Open,NaT,NaT,...,<NA>,999,<NA>,<NA>,Outstanding,E02000001,E01032739,201-3614,True,False
100001,201,City of London,6005,City of London School for Girls,11,Other independent school,1,Open,1920-01-01,NaT,...,Does not have boarders,999,<NA>,<NA>,,E02000001,E01000002,201-6005,False,True
100002,201,City of London,6006,St Paul's Cathedral School,11,Other independent school,1,Open,1939-01-01,NaT,...,Has boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6006,False,False
100003,201,City of London,6007,City of London School,11,Other independent school,1,Open,1919-01-01,NaT,...,Does not have boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6007,False,True
100005,202,Camden,1048,Thomas Coram Centre,15,Local authority nursery school,1,Open,NaT,NaT,...,<NA>,999,<NA>,<NA>,Outstanding,E02007115,E01000937,202-1048,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402468,679,Monmouthshire,5500,King Henry viii 3-19 School,30,Welsh establishment,1,Open,2023-09-01,NaT,...,<NA>,999,<NA>,<NA>,,999999999,999999999,679-5500,False,False
402469,681,Cardiff,2333,Ysgol Gynradd Groes-Wen Primary,30,Welsh establishment,1,Open,2023-09-01,NaT,...,<NA>,999,<NA>,<NA>,,W02000380,W01001729,681-2333,False,False
402470,668,Pembrokeshire,2398,Ysgol Bro Penfro,30,Welsh establishment,4,Proposed to open,2024-09-01,NaT,...,<NA>,999,<NA>,<NA>,,W02000140,W01000607,668-2398,False,False


In [20]:
cfo = pre_processing.build_cfo_data('data/cfo.xlsx')

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [21]:
academies = pre_processing.build_academy_data('data/academy_master_list.csv', 'data/gias_all_links.csv',
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, ks2, ks4, cfo, central_services)

In [22]:
#academies.to_csv('output/pre-processing/academies.csv')
academies[(academies["Trust UPIN"] == 126463) | (academies["Trust UPIN"] == 122824)].sort_index()[["Income_Total", "Total Expenditure", "Revenue reserve", "In year balance","Income_Total_CS", "Total Expenditure_CS", "Revenue reserve_CS", "In year balance_CS", "Trust Revenue reserve", "Total pupils in trust"]]

,Income_Total,Total Expenditure,Revenue reserve,In year balance,Income_Total_CS,Total Expenditure_CS,Revenue reserve_CS,In year balance_CS,Trust Revenue reserve,Total pupils in trust
URN,,,,,,,,,,
130912,1.063429e+07,12198000.0,0.0,-1383000.0,-180711.785079,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
131749,6.730960e+06,6372000.0,0.0,492000.0,-133040.250866,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
131895,7.753249e+06,8865000.0,0.0,-935000.0,-176751.380698,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
131896,7.507665e+06,6746000.0,0.0,906000.0,-144334.737433,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
132727,7.641824e+06,8086000.0,0.0,-284000.0,-160176.354956,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
...,...,...,...,...,...,...,...,...,...,...
147439,9.180770e+06,9805000.0,0.0,-462000.0,-162229.897969,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
147467,4.633162e+06,4688000.0,0.0,54000.0,-108837.779650,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0
148003,6.737096e+06,7524000.0,0.0,-681000.0,-105904.146775,2131000.0,51443000.0,-9966000.0,51443000.0,53415.0


Merge required census and cdc data to the maintained schools data set

In [23]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv','data/gias_all_links.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [24]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv')
#maintained_schools

In [25]:
all_schools = pd.concat([academies,maintained_schools])

## Federation Capture




In [26]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools.reset_index())

In [27]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

,URN,LAEstab,FederationName,FederationUid,Closed Date,Members
23,101997,3082040,The Hazel Wood Federation,1792,<NA>,"[101996, 101997]"
250,109923,8692231,Parsons Down Partnership of Schools,1616,<NA>,"[109875, 109923]"
251,109932,8692240,The Calcot Schools,1408,<NA>,"[109931, 109932]"
263,113223,8782446,South Hams Federation,1302,<NA>,"[113207, 113223, 113335, 113399]"
460,118442,8872439,The Bluebell Federation,1227,<NA>,"[118442, 118477]"
...,...,...,...,...,...,...
10109,121616,8153331,FST Primary Schools Federation,17673,<NA>,"[121324, 121492, 121616]"
10363,121602,8153289,BAWB Federation,16142,<NA>,"[121477, 121482, 121602]"
10366,121608,8153306,The Carlton & Faceby and Bilsdale Church of En...,1696,<NA>,"[121480, 121608]"
10580,102575,3142025,The Federation of Tolworth Infant and Nursery ...,1929,<NA>,"[102574, 102575]"


In [28]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

,LAEstab
350,8965204
459,8872403
461,8872494
597,9352002
1044,8693026
1970,9381110
2474,8132122
3870,8221009
4109,3547006
4916,8152189


# Budget Forcast Returns

In [29]:
aar_current_year = pre_processing.prepare_aar_data('data/academy_ar.xlsx')
cs_current_year = pre_processing.prepare_central_services_data("data/academy_ar.xlsx")
aar_2022 = pre_processing.prepare_aar_data('data/academy_ar_2022.xlsx')
cs_2022 = pre_processing.prepare_central_services_data("data/academy_ar_2022.xlsx")
aar_2021 = pre_processing.prepare_aar_data('data/academy_ar_2021.xlsx')
cs_2021 = pre_processing.prepare_central_services_data('data/academy_ar_2021.xlsx')

academies_current_year = pre_processing.build_academy_data('data/academy_master_list.csv', 'data/gias_all_links.csv',
                                              current_year, schools, census, sen, cdc, 
                                              aar_current_year, ks2, ks4, cfo, cs_current_year)[["Trust UPIN", "Company Registration Number", "Trust Revenue reserve", "Total pupils in trust"]]
academies_y1 = pre_processing.build_academy_data('data/academy_master_list.csv', 'data/gias_all_links.csv',
                                              current_year-1, schools, census, sen, cdc, 
                                              aar_2022, ks2, ks4, cfo, cs_2022)[["Trust UPIN", "Company Registration Number", "Trust Revenue reserve", "Total pupils in trust"]]
academies_y2 = None 

# pre_processing.build_academy_data('data/academy_master_list.csv', 'data/gias_all_links.csv',
#                                               current_year-2, schools, census, sen, cdc, 
#                                               aar_2021, ks2, ks4, cfo, cs_2021)

In [30]:
bfr, bfr_metrics = pre_processing.build_bfr_data(current_year, 'data/BFR_SOFA_raw.csv', 'data/BFR_3Y_raw.csv', academies_current_year, academies_y1, academies_y2)

In [36]:
bfr_metrics.sort_index()

,Category,value,Trust UPIN
Company Registration Number,,,
01532445,Revenue reserve as percentage of income,63.894325,<NA>
01532445,Slope,42428.571429,134845
01532445,Staff costs as percentage of income,73.091977,<NA>
01532445,Self generated income as percentage of income,12.915851,<NA>
01532445,Expenditure as percentage of income,110.469667,<NA>
...,...,...,...
13807225,Expenditure as percentage of income,39.682014,<NA>
13807225,Staff costs as percentage of income,26.631335,<NA>
13807225,Slope flag,0.000000,163851


### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')

There are 327 duplicates in the academies outputs, and 346 in the maintained (excluding federations)


academy_ar has 10444 entries, 148 of the urns in this list are duplicated, though they look to be schools which have changed from SAT to MAT

The academies_list containes duplicated LAEstabs due to schools transitioning between SAT / MAT etc.

maintained_schools has 10650 entries, 347 of which are nulls. These can just be dropped

There are a few duplicates in federation data as well: